In [ ]:
import pandas as pd
import io
import os
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
file1 = '/home/jovyan/Data/daily_pd5/Cleaned/pd_MX_2019_agg5_daily.csv'
df1 = pd.read_csv(file1)
df1 = df1.drop(columns=['grt', 'day', 'month'])
df1.columns = ['geohash_5', 'no_of_points', 'no_of_unique_users', 'local_date']
# df1.columns = ['geohash_5', 'no_of_points', 'no_of_unique_users', 'local_time', 'local_date']
df1 = df1[df1['local_date'].apply(lambda x: str(x).isdigit())]
df1['local_date'] = df1['local_date'].astype(int)
combined_df=df1

df1

In [ ]:
file2 = '/home/jovyan/Data/pd3/pd_mx2_2019_agg5_3h.csv'
df2 = pd.read_csv(file2)
# df1.columns = ['geohash_3', 'no_of_points', 'no_of_unique_users', 'local_date']
df2 = df2[df2['local_date'].apply(lambda x: str(x).isdigit())]
df2['local_date'] = df2['local_date'].astype(int)
df2

In [ ]:
combined_df = pd.concat([df1, df2], ignore_index=True)
combined_df

In [ ]:
# Parse the original datetime string and adjust the time
df1['local_time'] = pd.to_datetime(df1['local_time'], format='%Y%m%d %H:%M:%S')
df1['local_time'] = df1['local_time'].dt.strftime('%Y%m%d %H:%M:%S') + ' - ' + (df1['local_time'] + pd.Timedelta(hours=3)).dt.strftime('%H:%M:%S')

df1

In [ ]:
# import os
# import pandas as pd

# # Specify the folder containing the files
# folder_path = '/home/jovyan/Data/Fixing'

# # Get a list of all files in the folder
# file_list = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# # Initialize an empty DataFrame
# combined_df = pd.DataFrame()

# # Define the new column names
# new_columns = ['geohash_3', 'no_of_points', 'no_of_unique_users', 'local_date']

# # Loop through all files and append their content to the combined DataFrame
# for file in file_list:
#     file_path = os.path.join(folder_path, file)
#     df = pd.read_csv(file_path)
#     df.columns = new_columns  # Rename columns
#     combined_df = pd.concat([combined_df, df], ignore_index=True)

# combined_df

# Check duplicates

In [ ]:
# combined_df['geohash_3'] = combined_df['geohash_3'].astype(str)
combined_df['geohash_5'] = combined_df['geohash_5'].astype(str)

combined_df['no_of_points'] = combined_df['no_of_points'].astype(int)
combined_df['no_of_unique_users'] = combined_df['no_of_unique_users'].astype(int)
# combined_df['local_time'] = combined_df['local_time'].astype(str)
combined_df['local_date'] = combined_df['local_date'].astype(int)

combined_df = combined_df.drop_duplicates()
combined_df

# Check missing dates

In [ ]:
unique_values_count = combined_df["local_date"].unique()
all_dates = pd.date_range(start='2019-01-01', end='2019-12-31').strftime('%Y%m%d').astype(int)
all_dates = np.array(all_dates)
missing_dates = np.setdiff1d(all_dates, unique_values_count)

missing_dates

# Check for hours per day (for 3h)

In [ ]:
# Convert local_date to datetime objects
combined_df['local_date'] = pd.to_datetime(combined_df['local_date'], format='%Y%m%d')

# Function to parse the local_time column and extract the start hour
def parse_start_hour(time_str):
    try:
        start_time_str, _ = time_str.split(' - ')
        start_time = datetime.strptime(start_time_str, '%Y%m%d %H:%M:%S')
        return start_time.hour
    except ValueError:
        return None

combined_df['start_hour'] = combined_df['local_time'].map(parse_start_hour)

# Required hours to check
required_hours = set([0, 3, 6, 9, 12, 15, 18, 21])

# Group by local_date and check missing hours
missing_hours = {}

for date, group in combined_df.groupby('local_date'):
    present_hours = set(group['start_hour'])
    missing = required_hours - present_hours
    if missing:
        missing_hours[date] = sorted(list(missing))

# Convert missing hours dictionary to DataFrame for better readability
missing_hours_df = pd.DataFrame(list(missing_hours.items()), columns=['Date', 'Missing Hours'])

# Display the missing hours
print("Missing hours for each date:")
print(missing_hours_df)


# Check with figure

In [ ]:
daily_counts = combined_df['local_date'].value_counts().sort_index()

# Generate custom xticks starting from the first date and then incrementing by 7 days
start_date = daily_counts.index.min()
end_date = daily_counts.index.max()
custom_xticks = pd.date_range(start=start_date, end=end_date, freq='7D')

# Plot the results
plt.figure(figsize=(10, 6))
daily_counts.plot(kind='bar')
plt.title('Number of Rows for Each Day')
plt.xlabel('Date')
plt.ylabel('Number of Rows')
# plt.xticks(ticks=range(len(daily_counts)), labels=daily_counts.index.strftime('%Y-%m-%d'), rotation=90)
plt.gca().set_xticks([i for i, date in enumerate(daily_counts.index) if date in custom_xticks])
plt.tight_layout()
plt.show()

# Check Minimum and Maximum

In [ ]:
max_15_counts = daily_counts.nlargest(15)
max_15_counts

In [ ]:
min_5_counts = daily_counts.nsmallest(15)
min_5_counts

# Export

In [ ]:
# Clean up
combined_df['local_date'] = combined_df['local_date'].dt.strftime('%Y%m%d').astype(int)
combined_df = combined_df.sort_values('local_date').reset_index(drop = True)
combined_df

In [ ]:
combined_df

In [ ]:
# Save the combined DataFrame to a new CSV file
folder_path = '/home/jovyan/Data/daily_pd5/Cleaned/'
combined_df.to_csv(folder_path + 'pd_IN_2019_agg5_daily.csv', index=False)

# Check All Number (for R Kept Figure)

In [ ]:
%reload_ext sql
%config SqlMagic.autocommit=False
%config SqlMagic.autolimit=0
%config SqlMagic.autopandas=True
%config SqlMagic.displaylimit=200

In [ ]:
%sql trino://localhost:9090/cuebiq/

In [ ]:
import os
import time
# import geohash
import logging
import pandas as pd
from pathlib import Path
from datetime import datetime, timedelta
from sqlalchemy import create_engine

In [ ]:
# SQL engine
from trino.dbapi import connect 
from sqlalchemy import create_engine
import pandas as pd
import time

class TrinoEngine():
    def __init__(self):
        conn = connect(
            host="localhost",
            port=9090,
            catalog="cuebiq"
        )
        self.cur = conn.cursor()
        self.engine = create_engine("trino://localhost:9090/cuebiq/")
    
    def execute_statement(self, query:str) -> list:
        """
        Create and drop statements.
        """
        self.cur.execute(query)
        return self.cur.fetchall()
    
    def read_sql(self, query:str) -> pd.DataFrame: 
        """
        Select and insert into operations.
        """
        return pd.read_sql(query, self.engine)

sql_engine = TrinoEngine()

In [ ]:
schema_name = {'cda': 'cuebiq.paas_cda_pe_v3'}
pe_dl_table = f"{schema_name['cda']}.device_location_uplevelled"

In [ ]:
# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
# country_code = 'MX'
# country_code = 'IN'
# country_code = 'ID'
country_code = 'CO'

In [ ]:
pe_dl = sql_engine.read_sql(
    f"""
    SELECT 
        COUNT(*) AS total_no_of_points,
        COUNT(DISTINCT(cuebiq_id)) AS total_no_of_unique_users
    FROM {pe_dl_table}
    WHERE 
        country_code = '{country_code}' 
        AND event_zoned_datetime IS NOT NULL
        AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) IS NOT NULL
        -- AND DATE(TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s'))) BETWEEN date_parse('20191101', '%Y%m%d') AND date_parse('20191231', '%Y%m%d')
        AND DATE(TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s'))) BETWEEN date_parse('20200101', '%Y%m%d') AND date_parse('20201231', '%Y%m%d')
        AND lat <> 0
        AND lng <> 0
    """
)
pe_dl

 2019 (total_no_of_points total_no_of_unique_users)  2020  
CO: 126379332, 193424, 338744689, 201672
